## OpenAI APIキーの設定

In [1]:
import os

import openai
from dotenv import load_dotenv

load_dotenv()

# APIキーを設定
openai.api_key = os.getenv("OPENAI_API_KEY")

## RAGの作成
### LLMと埋め込みモデルの準備

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLMの準備
llm = ChatOpenAI(
    model="gpt-3.5-turbo", # モデル
    temperature=0, # ランダムさ
)

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings()

### ドキュメントの読み込みと分割

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ドキュメントの読み込み
loader = DirectoryLoader("./data/")
documents = loader.load()

# ドキュメントの分割
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, # ドキュメントサイズ (トークン数)
    chunk_overlap=0, # 前後でオーバーラップするサイズ
    separators=["\n\n"] # セパレーター
).split_documents(documents)
print(len(documents))

14


### VectorStoreの準備

In [4]:
from langchain_chroma import Chroma

# VectorStoreの準備
vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings,
)

### Retrieverの準備

In [5]:
# Retrieverの準備
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

### PromptTemplateの準備

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# PromptTemplateの準備
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "次のコンテキストのみを使用して、この質問に答えてください。\n\n{context}"),
        ("human", "{input}"),
    ]
)

### RAGチェーンの準備

In [7]:
from langchain_core.runnables import RunnablePassthrough

# RAGチェーンの準備
rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | llm
)

### 質問応答

In [8]:
# 質問応答
response = rag_chain.invoke("ギターヒーローの正体は？")
print(response.content)

後藤ひとり（ごとう ひとり）


### source付きRAGチェーンの準備

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# コンテキストのフォーマット
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Generationチェーンの準備
gemeration_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_template
    | llm
    | StrOutputParser()
)

# Retrieverチェーンの準備
retrieve_chain = (lambda x: x["input"]) | retriever

# source付きRAGチェーンの準備
rag_chain_with_source = RunnablePassthrough.assign(context=retrieve_chain).assign(
    answer=gemeration_chain
)

#### 質問応答

In [10]:
# 質問応答
rag_chain_with_source.invoke({"input": "ギターヒーローの正体は？"})

{'input': 'ギターヒーローの正体は？',
 'context': [Document(metadata={'source': 'data/bocchi.txt'}, page_content='後藤 ひとり (ごとう ひとり)：秀華高校に通う女子。桃色の髪を無造作に伸ばし、いつもジャージを身につけている。自他共に認める引きこもり一歩手前の「陰キャ」で、承認欲求が人一倍強いにもかかわらず、臆病な性格で人と接するのを極度に苦手としている。そのため、すぐに自分の世界に入って落ち込むという情緒不安定さを見せる。押し入れやダンボールに潜り込む癖があり、「完熟マンゴー仮面」を持ち歩いている。運動も勉強も苦手で、特に勉強はまじめに授業を受けているにもかかわらず、生来の要領の悪さから赤点ギリギリ。自分には何の取り柄もないのを痛感していたため、中学の頃に暗い性格の人間がバンドをやって人気者になったインタビューを読んで、ギターを始める。毎日練習したお陰でギターの腕前はプロ級になったが、結局、その腕前を披露する場に恵まれず、友達を作れないまま中学を卒業。現在は「ギターヒーロー」の名で動画配信している。ギタリストを探していた伊地知虹夏に誘われ、「結束バンド」に加入し、バンド活動を始める。バンド内ではギタリスト兼作詞を担当している。山田リョウに「ぼっち」の愛称を付けられて以降は、その愛称で呼ばれる。ギターの腕前はかなりのものだが、他人と合わせるセッションの経験が皆無なため、バンドではまともに演奏できないでいる。しかし徐々にバンドとして演奏できるようになり、文化祭ライブでは機材トラブルというアクシデントに見舞われながらも、アドリブでボトルネック奏法を行い、ライブを成功に導いた。なお、父親から借りたギターは文化祭で壊れたため、動画配信の広告収入で得たお金を使って、新たなギターを手に入れた。人見知りなため、美容院にもいけず、髪の毛を伸ばし放題で前髪で目をつねに隠している。それに加えて野暮ったいジャージ姿でいるため気づかれていないが、実は同性すら見とれるほどの美少女。黙って着飾っていれば「アイドル事務所に入れる」「ビジュアル担当」と言われるほどのポテンシャルを秘めているが、キメ顔は10秒しか持たず、ふだんの奇行がすべてを台無しにしている。'),
  Document(metadata={'

### 質問・正解・コンテキスト・回答の準備

In [11]:
# 質問
question = [
    "後藤ひとりの得意な楽器は？",
    "後藤ひとりの妹の名前は？",
    "後藤ひとりが加入したバンド名は？",
    "ギターヒーローの正体は？",
    "喜多郁代の髪の色は？",
    "伊地知虹夏が通う学校の名前は？",
    "山田リョウの趣味は？",
    "廣井きくりが所属するバンド名は？",
    "ライブハウス「STARRY」の店長の名前は？",
    "ぼっちちゃんが文化祭で披露した演奏法は？",
]

# 正解
ground_truth = [
    "ギター",
    "後藤ふたり",
    "結束バンド",
    "後藤ひとり",
    "赤",
    "下北沢高校",
    "廃墟探索と古着屋巡り",
    "SICKHACK",
    "伊地知星歌",
    "ボトルネック奏法",
]

# コンテキストと回答の生成
contexts = []
answer = []
for q in question:
    print("question:", q)
    response = rag_chain_with_source.invoke({"input": q})
    contexts.append([x.page_content for x in response["context"]])
    answer.append(response["answer"])

print("contexts:", contexts)
print("answer:", answer)

question: 後藤ひとりの得意な楽器は？
question: 後藤ひとりの妹の名前は？
question: 後藤ひとりが加入したバンド名は？
question: ギターヒーローの正体は？
question: 喜多郁代の髪の色は？
question: 伊地知虹夏が通う学校の名前は？
question: 山田リョウの趣味は？
question: 廣井きくりが所属するバンド名は？
question: ライブハウス「STARRY」の店長の名前は？
question: ぼっちちゃんが文化祭で披露した演奏法は？
contexts: [['後藤 ひとり (ごとう ひとり)：秀華高校に通う女子。桃色の髪を無造作に伸ばし、いつもジャージを身につけている。自他共に認める引きこもり一歩手前の「陰キャ」で、承認欲求が人一倍強いにもかかわらず、臆病な性格で人と接するのを極度に苦手としている。そのため、すぐに自分の世界に入って落ち込むという情緒不安定さを見せる。押し入れやダンボールに潜り込む癖があり、「完熟マンゴー仮面」を持ち歩いている。運動も勉強も苦手で、特に勉強はまじめに授業を受けているにもかかわらず、生来の要領の悪さから赤点ギリギリ。自分には何の取り柄もないのを痛感していたため、中学の頃に暗い性格の人間がバンドをやって人気者になったインタビューを読んで、ギターを始める。毎日練習したお陰でギターの腕前はプロ級になったが、結局、その腕前を披露する場に恵まれず、友達を作れないまま中学を卒業。現在は「ギターヒーロー」の名で動画配信している。ギタリストを探していた伊地知虹夏に誘われ、「結束バンド」に加入し、バンド活動を始める。バンド内ではギタリスト兼作詞を担当している。山田リョウに「ぼっち」の愛称を付けられて以降は、その愛称で呼ばれる。ギターの腕前はかなりのものだが、他人と合わせるセッションの経験が皆無なため、バンドではまともに演奏できないでいる。しかし徐々にバンドとして演奏できるようになり、文化祭ライブでは機材トラブルというアクシデントに見舞われながらも、アドリブでボトルネック奏法を行い、ライブを成功に導いた。なお、父親から借りたギターは文化祭で壊れたため、動画配信の広告収入で得たお金を使って、新たなギターを手に入れた。人見知りなため、美容院にもいけず、髪の毛を伸ばし放題で前髪で目をつねに

### データセットの準備

In [12]:
from datasets import Dataset

# データセットの準備
ds = Dataset.from_dict(
    {
        "question": question,
        "answer": answer,
        "contexts": contexts,
        "ground_truth": ground_truth,
    }
)

/Users/yd03/dev/github/langchain-tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 評価の実行

In [13]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# 評価の実行
result = evaluate(ds, [faithfulness, answer_relevancy, context_precision, context_recall])
print(result)

Evaluating: 100%|██████████| 40/40 [00:14<00:00,  2.68it/s]


{'faithfulness': 0.5000, 'answer_relevancy': 0.8811, 'context_precision': 1.0000, 'context_recall': 1.0000}


## ragas score
### generation

faithfulness
- 回答がコンテキストに対してどれだけ関連しているか

answer_relevancy
- 回答が質問に対してどれだけ関連しているか
### retrieval

context_precision
- コンテキストが質問に対してどれだけ関連しているか

context_recall
- コンテキストが正解に対してどれだけ関連しているか